In [1]:
data_dir = "/scratch-shared/TANJ0303/datasets/"

In [2]:
!export HF_DATASETS_CACHE=data_dir

In [3]:
import sys
import os

def is_colab_env():
    return "google.colab" in sys.modules

def mount_google_drive(drive_dir="/content/drive/", repo_dir="MyDrive/repositories/deepfake-detection"):
    # mount google drive
    from google.colab import drive
    drive.mount(drive_dir)

    # change to correct working directory
    import os
    os.chdir(f"{drive_dir}{repo_dir}")
    print(os.listdir()) # verify content

def resolve_path(levels_deep=3):
    if is_colab_env():
        mount_google_drive()
        return ""
    else:
        # Get the directory of the current script
        current_dir = os.path.dirname(os.path.abspath('__file__'))

        # Construct the path to the parent directory
        for i in range(levels_deep):
            current_dir = os.path.dirname(current_dir)

        # Add the parent directory to sys.path
        sys.path.append(current_dir)
        print(sys.path)
        return current_dir

proj_dir = resolve_path()

['/home/FYP/tanj0303/.conda/envs/df-env/lib/python310.zip', '/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10', '/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/lib-dynload', '', '/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/site-packages', '/home/FYP/tanj0303']


In [4]:
# import local config
import config

In [5]:
# import library dependencies
import numpy as np

In [6]:
# pytorch
import torch
import pytorch_lightning as L

In [7]:
# import local dependencies
from src.adapters.datasets.wilddeepfake import WildDeepfakeDataModule
from src.models.resnet import ResNetClassifier

In [8]:
model_id = "fft_magnitude_phase_resnet18"
model_checkpoint_dir = f"{proj_dir}/{config.CHECKPOINTS_DIR}/{model_id}"

In [9]:
from src.transforms.frequency import get_transforms
fft_mag_phase_transforms = get_transforms("fft_mag_phase")

In [10]:
# max_samples = 800_000  # For quick development, remove for full dataset
batch_size = 32
num_workers = 20
seed = config.SEED

# Set seeds for reproducibility
torch.manual_seed(seed)
np.random.seed(seed)

# Determine device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# define datamodule
wilddeepfake_data_module = WildDeepfakeDataModule(
    batch_size=batch_size,
    num_workers=num_workers,
    # max_samples=max_samples,
    seed=seed,
    transforms=fft_mag_phase_transforms,
    dataset_cache_dir=data_dir
)

Using device: cuda


In [11]:
# define early stopper
early_stop_callback = L.callbacks.EarlyStopping(
    monitor="val_loss",       # metric to track
    patience=3,               # epochs to wait for improvement
    mode="min",               # "min" because we want val_loss to decrease
    verbose=True
)

In [12]:
# define ligntning checkpoint
best_loss_checkpoint = L.callbacks.ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

In [13]:
# define model
deepfake_detector = ResNetClassifier(in_channels=2, freeze_features=False)

In [14]:
# define trainer
max_epochs = 100
trainer = L.Trainer(
    devices=1,
    callbacks=[early_stop_callback, best_loss_checkpoint],
    default_root_dir=model_checkpoint_dir,
    log_every_n_steps=100,
    profiler="simple", # track time taken
    max_epochs=max_epochs,
  )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [15]:
# train model
trainer.fit(deepfake_detector, datamodule=wilddeepfake_data_module)

Resolving data files:   0%|          | 0/963 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/157 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/124 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset loaded. Processing samples...


...

Metric val_loss improved by 0.146 >= min_delta = 0.0. New best score: 0.304


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.304. Signaling Trainer to stop.
FIT Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                             

In [16]:
# test dataset on unseen samples
trainer.test(deepfake_detector, datamodule=wilddeepfake_data_module)
# trainer.test(deepfake_detector, test_loader)

Resolving data files:   0%|          | 0/963 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/157 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/124 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset loaded. Processing samples...


...

Extracted labels. Generating train/val split...
Train samples: 811549, Val samples: 202888, Test samples: 165662


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6732684373855591
        test_auc            0.7039417624473572
         test_f1             0.744091272354126
        test_loss           0.6186311841011047
        test_prec           0.7529807090759277
        test_rec            0.7354092597961426
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


TEST Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                                	|  -      

[{'test_acc': 0.6732684373855591,
  'test_prec': 0.7529807090759277,
  'test_rec': 0.7354092597961426,
  'test_f1': 0.744091272354126,
  'test_auc': 0.7039417624473572,
  'test_loss': 0.6186311841011047}]

In [17]:
# view metrics from previous runs
%reload_ext tensorboard
%tensorboard --logdir=$model_checkpoint_dir

ModuleNotFoundError: No module named 'tensorboard'